Progress Report:
- DONE: Feb 14: can output a file written with comments but error in delimiters, every character delimited by ','
    Also, Python error about being unable to transcribe unicode for a certain string about 3 secs into loop in get_comments()

In [2]:
import json
import pandas as pd
import requests
import numpy as np
import pandas as pd
import csv

In [2]:
# set parameters for request

subreddit = 'asianamerican'
limit = 100
timeframe = 'all' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top

### get_reddit(): returns .json object

In [2]:
def get_reddit(subreddit,listing,limit,timeframe):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}'
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'})
    except:
        print('An Error Occured')
    return request.json()

In [3]:
r = get_reddit(subreddit, listing, limit, timeframe)

NameError: name 'subreddit' is not defined

In [6]:
def get_post_titles(r):
    '''
    Get a List of post titles
    '''
    posts = []
    for post in r['data']['children']:
        x = post['data']['title']
        posts.append(x)
    return posts

In [7]:
titles = get_post_titles(r)

In [8]:
print(len(titles))

100


In [9]:
print(titles[0:5])

['NBA All-Star Damian Lillard wears ""Stop Asian Hate" shirt while sitting out tonight\'s game', '“Fuck you! We will stop the Hate!” NBA star Baron Davis stands against anti-Asian racism.', 'Accurate', "My friend's mother was one of the victims of Anti-Asian shooting that recently occurred. The gofundme already lays out their situation but she was a single mother working hard to support her kids through school and the pandemic. Any contributions would mean the world", 'Megan Thee Stallion donates $50K to AAPI legal advocacy group after Atlanta spa shootings']


In [21]:
# print keys/elements of a Reddit object (in this case, submission (t3))
list_red_objs = list(search['data']['children'][0]['data'].keys())

# important keys: (1) id (Reddit thing ID), (2) title, (3) ups (upvotes), 
# (4) downs (downvotes), (5) score (not sure), (6) likes (not sure), (7) num_comments)

['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'subreddit_type', 'ups', 'total_awards_received', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'post_hint', 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html

In [11]:
# What categories do I need from each post?
## Comments
## 

#print(r['data']['children'])
dict_ = pd.DataFrame.from_dict(r['data']['children'][0])
dict_[0:40]

,kind,data
all_awardings,t3,[]
allow_live_comments,t3,True
approved_at_utc,t3,None
approved_by,t3,None
archived,t3,True
author,t3,Shiggy_O
author_flair_background_color,t3,None
author_flair_css_class,t3,None
author_flair_richtext,t3,[]
author_flair_template_id,t3,None


### get_search()
- gets the .json file of a Reddit search

In [3]:
# Corrected request url
#https://www.reddit.com/r/asianamerican/search/?q=affirmative%20action&restrict_sr=1&sort=top

def get_search(subreddit,query,limit,timeframe,sort,restrict_sr):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/search.json?q={query}&restrict_sr={restrict_sr}&limit={limit}&t={timeframe}&sort={sort}'
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'})
    except:
        print('An Error Occured')
    return request.json()

### Get top 10 post (submission) IDs from Search

In [4]:
def get_top_n_submission_info(r, n):
    '''
    Get a List of post titles
    r: .json object
    n: num of top posts
    # important keys: (1) id (Reddit thing ID), (2) title, (3) ups (upvotes), 
    # (4) downs (downvotes), (5) score (not sure), (6) likes (not sure), (7) num_comments)

    Returns: pandas Dataframe object
    '''
    # create 3 empty arrays of length n
    id_arr = np.empty(n, dtype="S10")
    up_arr = np.empty(n)
    down_arr = np.empty(n)
    score_arr = np.empty(n)
    likes_arr = np.empty(n)
    num_com_arr = np.empty(n)
    
    for i, post in enumerate(r['data']['children']):
        id_arr[i] = post['data']['id']
        up_arr[i] = post['data']['ups']
        down_arr[i] = post['data']['downs']
        score_arr[i] = post['data']['score']
        likes_arr[i] = post['data']['likes']
        num_com_arr[i] = post['data']['num_comments']

    # create Dataframe
    df = pd.DataFrame({'id': id_arr, 'upvotes': up_arr, 'downvotes': down_arr, 'score': score_arr, 'likes': likes_arr, 'num_comments': num_com_arr})


    return df

#### Driver file

In [28]:
# get_search() params
sr = 'asianamerican'
q = 'affirmative action'
limit = 150
timeframe = 'all'
sort = 'top'
restrict_sr = 1

# variable for array of titles
titles = None

search = get_search(subreddit=sr,query=q, limit=limit, timeframe=timeframe, sort=sort, restrict_sr=restrict_sr)
df = get_top_n_submission_info(search, limit)

In [30]:
df.tail(60)

,id,upvotes,downvotes,score,likes,num_comments
90,b'3rkhqd',5.0,0.000000e+00,5.000000e+00,NaN,0.0
91,b'2c8iu1',8.0,0.000000e+00,8.000000e+00,NaN,21.0
92,b'4eb7qj',4.0,0.000000e+00,4.000000e+00,NaN,7.0
93,b'5cw7uu',6.0,0.000000e+00,6.000000e+00,NaN,11.0
94,b'24hmx8',6.0,0.000000e+00,6.000000e+00,NaN,31.0
95,b'4lqzqq',5.0,0.000000e+00,5.000000e+00,NaN,83.0
96,b'd6767e',6.0,0.000000e+00,6.000000e+00,NaN,33.0
97,b'esbwk7',6.0,0.000000e+00,6.000000e+00,NaN,1.0
98,b'6ek03l',6.0,0.000000e+00,6.000000e+00,NaN,0.0
99,b'4q20ib',3.0,0.000000e+00,3.000000e+00,NaN,8.0


In [6]:
# get column of t3 object submission IDs
id_arr = df['id'].to_numpy()
id_arr = id_arr.astype('str')

print(id_arr[:15])

['14m8mf4' 'hqic5b' '9nk0do' 'sejv5m' 'h9tuc2' 'sbrca3' 'cwk0vc' '8ajkwc'
 '8usex7' 'rjy8ut' '7481yg' '5btr2j' '87ru7q' 'pvb846' 'jny9p4']


## Get comments

### Using Reddit API

In [17]:
# Don't use below code - use PRAW instead
def get_comments(sr, post_id, context, limit, showedits, showmedia, showmore, showtitle, theme, threaded, truncate, sort):
    try:
        base_url = f'https://www.reddit.com/r/{sr}/comments/{post_id}?context={context}&limit={limit}&showedits={showedits}&showmore={showmore}&showmedia={showmedia}&showtitle={showtitle}&sort={sort}&theme={theme}&threaded={threaded}&truncate={truncate}'
        request = requests.get(base_url, headers = {'User_agent': 'yourbot'})
        return request.json()
    except:
        print('An Error Occured')

In [11]:
sr = 'asianamerican'
context = 5 # int: 0-8
limit = 100
showedits = False
showmore = False
showmedia = False
showtitle = False
sort = 'confidence' # one of (confidence, top, new, controversial, old, random, qa, live)
theme = 'default'
threaded = False
truncate = 25

In [42]:
comments = get_comments(sr=sr,
                        post_id=post_id,
                        context=context,
                        limit=limit,
                        showedits=showedits,
                        showmore=showmore,
                        showtitle=showtitle,
                        showmedia=showmedia,
                        sort=sort,
                        theme=theme,
                        threaded=threaded,
                        truncate=truncate)

An Error Occured


### Using PRAW Instead of Reddit API

In [7]:
import praw

# need to take secret and password off of Github in the future

reddit = praw.Reddit(
    client_id="iaKUGuo77BwJYHElyAo1Jg",
    client_secret="19bSH4KJpPJB2WFeYhUe--jk4IKeHQ",
    password='Ehdwn1928K!',
    user_agent="Comment Extraction u/dlehdwn",
    username='dlehdwn',
)

In [8]:
print(reddit.user.me())

dlehdwn


In [71]:
#enable read-only mode
#reddit.read_only = True

In [23]:
def get_comments(praw_obj, id_arr, filename):
    '''
    praw_obj: PRAW Reddit object
    id_arr: array of Reddit t3 IDs
    '''
    # manual count of comments while traversing thru CommentTree to compare to actual number of comments
    num_comments = 0

    #open a file to write to
    with open(filename, 'w', newline='', encoding="utf-8") as f:

        # writer
        writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)

        for id in id_arr: #try just first 2 submission ids
            sub = praw_obj.submission(id)

            # get all comments
            sub.comments.replace_more(limit=None, threshold=1) #threshold - min num of comments needed to traverse into "More comments" object
            for comment in sub.comments.list():

                # get comment text
                body = comment.body

                # strip comment of delimiter (|)
                #body.replace('|','')
                #body = body.strip() + '|'

                # get username of comment author
                if comment.author is not None:
                    username = comment.author.name
                    flair = comment.author_flair_text
                    writer.writerow([username, flair, body])
                
                else:
                    #print('comment.author is None')
                    #print(f'body: {body}\n')
                    writer.writerow(['','',body])     

                num_comments+=1
    f.close()

    # return num of comments written
    return num_comments



In [24]:
num_comments_collected = get_comments(reddit, id_arr, 'top_100_post_comments_user_flair.txt') #takes a min to run

In [22]:
# num of actual total comments
print(df['num_comments'].sum())

# num of collected comments
print(num_comments_collected)

3699.0
3623


#### Num of comments collected
- Num of collected comments in top 100 posts: 3623
- Num of actual total comments in top 100 posts: 3699

In [23]:
submission = reddit.submission("14m8mf4")

In [24]:
# print top level comments
for top_level_comment in submission.comments:
    print(top_level_comment.body)

Thanks to everyone who engaged in insightful and respectful discourse about the news. 

This thread is now locked for comments.
I would prefer using a process that takes into account poverty instead. 

The first generation of my family that came to America was *painfully* poor and everyone showed up with neither money nor education. They worked in kitchens and laundromats. Notice a lot of people in bigger reddit boards talking shit about the "Chinese billionaire" boogeyman (fearmongering like this also erases the less visible Asian races who came to America as refugees and reduces all Asians to a monolithic "rich Asian stereotype") and how this will only help them. The Chinese people I know were not coming to America with bags of cash.
u/Tungsten_, Thanks for creating a section just to discuss this. When I read the news I immediately went searching for a forum where folks might have civil discourse on this topic.

Just had a few comments/questions:

1. Has anyone come across seemingly 

In [29]:
submission.comments.replace_more(limit=None)
for comment in submission.comments.list():
    print(comment.body)

Thanks to everyone who engaged in insightful and respectful discourse about the news. 

This thread is now locked for comments.
I would prefer using a process that takes into account poverty instead. 

The first generation of my family that came to America was *painfully* poor and everyone showed up with neither money nor education. They worked in kitchens and laundromats. Notice a lot of people in bigger reddit boards talking shit about the "Chinese billionaire" boogeyman (fearmongering like this also erases the less visible Asian races who came to America as refugees and reduces all Asians to a monolithic "rich Asian stereotype") and how this will only help them. The Chinese people I know were not coming to America with bags of cash.
u/Tungsten_, Thanks for creating a section just to discuss this. When I read the news I immediately went searching for a forum where folks might have civil discourse on this topic.

Just had a few comments/questions:

1. Has anyone come across seemingly 